In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
from datetime import timedelta
import regex as re
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import pyodbc
from sqlalchemy import create_engine

##### Os dados do projeto são de uma empresa real, as informações privadas serão ocultas
---
##### Os procedimentos estão bem descritos até o final da sessão 4. A partir da sessão 5, são realizadas as mesmas análises das sessões 3 e 4, porém agrupadas pelo canal de vendas.

# Integração Python / SQL

In [3]:
# Criando variáveis com as informações da empresa para acesso ao servidor SQL Server
server = ##### 
database = ####
username = #####
password = #####

In [4]:
# utilizando a biblioteca pyodbc para conectar com o servido e baixar os dados
cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [5]:
cursor = cnxn.cursor()

cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0]) 
    row = cursor.fetchone()

Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 
	Sep 24 2019 13:48:23 
	Copyright (C) 2019 Microsoft Corporation
	Express Edition (64-bit) on Windows Server 2016 Standard 10.0 <X64> (Build 14393: ) (Hypervisor)



In [6]:
# A informação de Canal de venda do pedido não está no banco de dados, porém, é uma informação atrelada ao vendedor
# Criando uma query para criar a coluna 'MÉDIO_CANAL' no DF importado da base de dados
query_ecm_full = '''
select *,

CASE
WHEN Vendedor = 'AMERICANAS' THEN 'B2W'
WHEN Vendedor = 'SHOPTIME' THEN 'B2W'
WHEN Vendedor = 'SUBMARINO' THEN 'B2W'
WHEN Vendedor = 'AMAZON' THEN 'AMAZON'
WHEN Vendedor = 'CARREFOUR' THEN 'MKTPLACE'
WHEN Vendedor = 'CONECTA_LA' THEN 'CONECTA LA'
WHEN Vendedor = 'KABUM' THEN 'KABUM'
WHEN Vendedor = 'LOJA_VIVO' THEN 'MKTPLACE'
WHEN Vendedor = 'MADEIRA MADEIRA' THEN 'MKTPLACE'
WHEN Vendedor = 'NOVO MUNDO' THEN 'MKTPLACE'
WHEN Vendedor = 'OLIST' THEN 'OLIST'
WHEN Vendedor = 'RAPPI' THEN 'MKTPLACE'
WHEN Vendedor = 'SHOPEE' THEN 'SHOPEE'
WHEN Vendedor = 'VIA VAREJO (CNOVA)' THEN 'VIA VAREJO (CNOVA)'
WHEN Vendedor = 'ECM EMPRESAS' THEN 'ECM EMPRESAS'
WHEN Vendedor = 'KAZUK' THEN 'KAZUK'
WHEN Vendedor = 'KZK ML' THEN 'KAZUK'
WHEN Vendedor = 'MAGAZINE LUIZA' THEN 'MAGALU'
WHEN Vendedor = 'MERCADO LIVRE' THEN 'MERCADO LIVRE'
WHEN Vendedor = 'ML FULL' THEN 'MERCADO LIVRE FULL'
WHEN Vendedor = 'BACKUP' THEN 'OUTROS'
WHEN Vendedor = 'CD PRESENCIAL' THEN 'ESPACO ECMSCREEN'
WHEN Vendedor = 'INDICAÇÃO' THEN 'OUTROS'
WHEN Vendedor = 'ALICE LEAL' THEN 'REVENDA'
WHEN Vendedor = 'ALINE DIAS' THEN 'REVENDA'
WHEN Vendedor = 'BRUNO T' THEN 'REVENDA'
WHEN Vendedor = 'FERNANDO M' THEN 'REVENDA'
WHEN Vendedor = 'FLAVIO' THEN 'REVENDA'
WHEN Vendedor = 'JOSIANE' THEN 'REVENDA'
WHEN Vendedor = 'LEANDRO' THEN 'REVENDA'
WHEN Vendedor = 'WESLEY' THEN 'REVENDA'
WHEN Vendedor = 'REENVIO' THEN 'RMA'
WHEN Vendedor = 'RMA' THEN 'RMA'
WHEN Vendedor = 'MAGENTO' THEN 'MAGENTO'
WHEN Vendedor = 'PJ SITE' THEN 'SITE'
WHEN Vendedor = 'REVENDA SITE' THEN 'SITE'
WHEN Vendedor = 'SITE' THEN 'SITE'
END as 'MÉDIO_CANAL'


from base_hist_custo_fixo

'''
ecm_full = pd.read_sql_query(query_ecm_full, cnxn)

# Limpezaz de dados

In [7]:
# Na query não vem o TIPO_VENDA (pode ser vendas ou devolução) e nem a SEMANA
# Existe uma coluna devolução com um valor. Colocar apenas as linhas em que 'Devolução' == 0 para ter certeza que é uma venda
# Criar e adicionar a coluna Semana
mask_dev = ecm_full['Devolução'] == 0
ecm_full = ecm_full[mask_dev]
ecm_full['SEMANA'] = ecm_full['Data Emissão'].apply(lambda x: x.strftime("%U"))

In [8]:
# Selecionado as colunas que vou utilizar da base de dados
ecm_vendas = ecm_full[['Tipo', 'Data Emissão',
                       'MÉDIO_CANAL', 'Código', 'Volume / Mês', 'Vr Unitário', 'SEMANA', 'Margem Contribuição Sem Imposto (R$)']]

# Na coluna tipo, existem pedidos de teste para testar o sistema. Vou filtrar apenas para os pedidos reais
mask_tipo = ecm_full['Tipo'] == 'Real'
ecm_vendas = ecm_vendas[mask_tipo]
# Transformando o preço em float
ecm_vendas['Vr Unitário'] = ecm_vendas['Vr Unitário'].map(float)
# Mudando o  nome da data do pedido
ecm_vendas = ecm_vendas.rename(columns={'Data Emissão' : 'data'})
# dropando a coluna Tipo
ecm_vendas = ecm_vendas.drop(columns = ['Tipo'])

# Análise de Vendas:
##### O objetivo aqui é comparar o resultados de vendas da semana passada com o período de 90 dias

## Análise geral do histórico de vendas nos últimos 90 dias

   ### Principais DF gerados:
        - 'ecm_delta_calc' = Todos os pedidos dentro do período
        - '_df_base' = baselines, média de vendas e preço médio de cada produto no período

In [11]:
# Gerando o DF do período de 90 dias para base de informações
hoje = datetime.today()
delta_0 = timedelta(days = 8)
semana_antes = hoje - delta_0
periodo = 90
delta_1 = timedelta(days = periodo)
min_periodo = semana_antes - delta_1

mask_periodo = ecm_vendas['data'] > min_periodo
ecm_delta_calc = ecm_vendas[mask_periodo]
mask_periodo_semana = ecm_delta_calc['data'] < semana_antes
ecm_delta_calc = ecm_delta_calc[mask_periodo_semana]

# agrupando os pedidos para a somatória das vendas e o preço médio no período
ecm_ref = ecm_delta_calc.groupby(by = ['Código', 'SEMANA'], as_index=False).agg(
    vendas_sum = pd.NamedAgg(column = 'Volume / Mês', aggfunc = 'sum'),
    vlr_mean = pd.NamedAgg(column = 'Vr Unitário', aggfunc = 'mean'))

# criando uma base de dados a partir do período com as seguintes informações:
    # SKU
    # média de vendas por semana
    # preço médio
    # base esperada de vendas = média de vendas no período
    # base máxima no período = média + 1 desvio padrão
lt_codigos = list()
lt_media_vendas_semana = list()
lt_preco_med_periodo = list()
lt_baseline_min = list()
lt_baseline_max = list()

# Criando um loop que percorra todos os códigos e guarde as informações em listas
for cod in ecm_ref['Código'].unique():
    med_vend = ecm_ref[ecm_ref['Código'] == cod]['vendas_sum'].mean()
    if ecm_ref[ecm_ref['Código'] == cod]['vendas_sum'].std() > 0:
        baseline_max = (ecm_ref[ecm_ref['Código'] == cod]['vendas_sum'].mean() + 
                        ecm_ref[ecm_ref['Código'] == cod]['vendas_sum'].std())
    else:
        baseline_max = (ecm_ref[ecm_ref['Código'] == cod]['vendas_sum'].mean())
    med_pc = ecm_ref[ecm_ref['Código'] == cod]['vlr_mean'].mean()
    lt_codigos.append(cod)
    lt_baseline_max.append(baseline_max)
    lt_media_vendas_semana.append(med_vend)
    lt_preco_med_periodo.append(med_pc)

# juntando as listas para criar o DF com as informações necessárias do período para comparação com a semana atual
df_base = pd.DataFrame({'código': lt_codigos, 'base_alvo': lt_media_vendas_semana, 
                        'base_max': lt_baseline_max, 'preço_médio_90dias': lt_preco_med_periodo})
# Organizando as informações
df_base['preço_médio_90dias'] = df_base['preço_médio_90dias'].round(2)
df_base['base_max'] = df_base['base_max'].round(0)
df_base['base_alvo'] = df_base['base_alvo'].round(0)
df_base = df_base.sort_values(by = 'base_alvo', ascending = False)

## Análise de Vendas da Semana Passada
### Principais DF gerados:
    - 'ecm_passada' = todos os pedidos feitos na semana passada
    - 'teste' = informações de vendas da semana passada agrupados pelo SKU
    - 'relatorio_t' = relatório agrupado das informações da semana passada com o período

In [12]:
# Selecionando as data da semana passada
delta_semana = timedelta(days = 8)
semana = hoje - delta_semana

# Criando o DF com cada pedido da semana passada
mask_semana = (ecm_vendas['data'] > semana)
ecm_semana = ecm_vendas[mask_semana]
time_fds = timedelta(days = 2)
fds = hoje - time_fds
mask_fds = ecm_semana['data'] < fds
ecm_semana = ecm_semana[mask_fds]
ecm_semana = ecm_semana.reset_index()

In [ ]:
# agrupando informações pelo SKU para calcular:
    # total de vendas na semana passada
    # Média de preço
    # margem de contribuição total
teste = ecm_semana.groupby(by = ['Código'], as_index=False).agg(
    vendas_semana_passada = pd.NamedAgg(column = 'Volume / Mês', aggfunc = 'sum'),
    média_preço_semana = pd.NamedAgg(column = 'Vr Unitário', aggfunc = 'mean'),
    MC_total = pd.NamedAgg(column = 'Margem Contribuição Sem Imposto (R$)', aggfunc = 'sum'))

# Mudando o nome das colunas para aparecer a semana do ano em que a análise foi feita
teste = teste.rename(columns = {'vendas_semana_passada' : 'vendas_s' + ecm_semana['SEMANA'][0],
                                'média_preço_semana' : 'R$_médio_s' + ecm_semana['SEMANA'][0],
                                'MC_total' : 'MC_total_s' + ecm_semana['SEMANA'][0]})


# Criando um DF juntando as informações do período com o período
teste = teste.rename(columns={'Código' : 'código'})
relatorio_t = teste.merge(df_base, how = 'outer', on = 'código')


# Classificando os resultados de vendas da semana
relatorio_t['RESULTADO (vendas)'] = np.where(relatorio_t[relatorio_t.columns[1]] >= relatorio['base_max'], 
                                       'ACIMA', np.where(
                                       relatorio_t[relatorio_t.columns[1]] <= relatorio['base_alvo'],
                                       'ABAIXO', np.where(
                                           ((relatorio_t[relatorio_t.columns[1]] < relatorio['base_max'])
                                            &
                                           (relatorio_t[relatorio_t.columns[1]] > relatorio['base_alvo'])),
                                           'ESPERADO', 'N/A')))

# Criando coluna com a diferença % do preço na semana com o período
relatorio_t[relatorio_t.columns[2]] = relatorio_t[relatorio_t.columns[2]].round(2)
relatorio_t['Dif. % R$_médio'] = ((relatorio_t[relatorio_t.columns[2]] / relatorio_t['preço_médio_90dias']) - 1).round(2)

# criando coluna com a MC de cada SKU na semana
relatorio_t['MC_unidade'] = relatorio_t[relatorio_t.columns[3]] / relatorio_t[relatorio_t.columns[1]]
relatorio_t = relatorio_t.rename(columns = {'MC_unidade' : 'MC/unidade_s' + ecm_semana['SEMANA'][0]}).round(2)

# Calculando MC do período
    # aqui estou agrupando por SKU e calculando o total de vendas e a Margem de contribuição total
MC_unid_periodo = ecm_delta_calc.groupby(by = 'Código', as_index=False).agg(
    total_vendas_periodo = pd.NamedAgg(column = 'Volume / Mês', aggfunc = 'sum'),
    total_mc = pd.NamedAgg(column = 'Margem Contribuição Sem Imposto (R$)', aggfunc = 'sum'))
    
    # aqui estou dividindo o total de vendas pela margem de contribuição total 
    # Com isso encontro a margem de contribuição unitária
MC_unid_periodo['MC_unitária'] = (MC_unid_periodo['total_mc'] / MC_unid_periodo['total_vendas_periodo']).round(2)
mc_sku_unid = MC_unid_periodo[[ 'Código', 'MC_unitária']]
mc_sku_unid = mc_sku_unid.rename(columns={'Código' : 'código'})

# juntando o relatorio_t com a margem de contribuição dos SKUs
relatorio_t = relatorio_t.merge(mc_sku_unid, how = 'outer', on = 'código')
relatorio_t = relatorio_t.rename(columns = {'MC_unitária' : 'MC/unidade_período'})

# Calculando a diferença de margem de contribuição de cada SKU com a do período
relatorio_t['Dif_MC/unid'] = relatorio_t[relatorio_t.columns[9]] - relatorio_t['MC/unidade_período']

# Calculando a média da MC por semana no período
    # Aqui calculo o total da margem de contribuição por semana
mc_semana = ecm_delta_calc.groupby(by = ['Código', 'SEMANA'], as_index=False).agg(
    MC_semana = pd.NamedAgg(column = 'Margem Contribuição Sem Imposto (R$)', aggfunc = 'sum'))

    # Aqui calculo a quantidade de semanas no período e a Margem de contribuição total de cada SKU
mc_semana_periodo = mc_semana.groupby(by = 'Código', as_index = False).agg(
    qnt_semanas = pd.NamedAgg(column = 'SEMANA', aggfunc = 'count'),
    mc_total_per = pd.NamedAgg(column = 'MC_semana', aggfunc = 'sum'))

    # Aqui eu divido a margem de contribuição total pela quantidade de semanas
    # assim tenho a média da mergem de contribuição por semana de cada SKU
mc_semana_periodo['média_mc/semana'] = (mc_semana_periodo['mc_total_per'] / mc_semana_periodo['qnt_semanas']).round(2)
media_mc_periodo = mc_semana_periodo[[ 'Código', 'média_mc/semana']]

# aqui eu junto as informações da margem de contribuição com o relatório principal
media_mc_periodo = media_mc_periodo.rename(columns = {'Código' : 'código'})
relatorio_t = relatorio_t.merge(media_mc_periodo, how = 'outer', on = 'código')
relatorio_t = relatorio_t.rename(columns = {'média_mc/semana' : 'MC/semana_período'})

# Aqui eu calculo a diferença da margem de contribuição da semana com a média do período
relatorio_t['Dif. MC/semana'] = relatorio_t[relatorio_t.columns[3]] - relatorio_t['MC/semana_período']

# Gerando relatório final
    - relatorio_final = No final deixei um modelo do relatório final

In [14]:
# Gerando 'relatorio_final'
relatorio_t = relatorio_t.rename(columns={'preço_médio_90dias' : 'R$_médio_periodo'})

relatorio_final = relatorio_t [[ 'código',  'base_alvo', 'base_max', relatorio_t.columns[1], 
                                'R$_médio_periodo', relatorio_t.columns[2], 
                               'MC/unidade_período', relatorio_t.columns[9],
                               'MC/semana_período', relatorio_t.columns[3],
                               'Dif. % R$_médio', 'Dif_MC/unid', 'Dif. MC/semana', 'RESULTADO (vendas)']]

# Classificando os resultados da margem de contribuição por unidade de SKU
relatorio_final['RESULTADO (MC/unid)'] = np.where(relatorio_final['Dif_MC/unid'] > 0, 'POSITIVO',
                                                  np.where(
                                        relatorio_final['Dif_MC/unid'] <= 0, 'NEGATIVO', 'N/A'))

# Classificando os resultados da margem de contribuição total dos SKUs
relatorio_final['RESULTADO (MC_total/semana)'] = np.where(relatorio_final['Dif. MC/semana'] > 0, 'POSITIVO', np.where(
                                        relatorio_final['Dif. MC/semana'] <= 0, 'NEGATIVO', 'N/A'))

<ipython-input-14-04aa7188ad3e>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relatorio_final['RESULTADO (MC/unid)'] = np.where(relatorio_final['Dif_MC/unid'] > 0, 'POSITIVO',
<ipython-input-14-04aa7188ad3e>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relatorio_final['RESULTADO (MC_total/semana)'] = np.where(relatorio_final['Dif. MC/semana'] > 0, 'POSITIVO', np.where(


In [15]:
# organizando o relatório em ordem que os produtos com mais queda na margem de contribuição apareçam primeiro
relatorio_final = relatorio_final.sort_values(by = 'Dif. MC/semana')

In [16]:
# Visualização do relatório final para análise do setor comercial da empresa.
relatorio_final

,código,base_alvo,base_max,vendas_s17,R$_médio_periodo,R$_médio_s17,MC/unidade_período,MC/unidade_s17,MC/semana_período,MC_total_s17,Dif. % R$_médio,Dif_MC/unid,Dif. MC/semana,RESULTADO (vendas),RESULTADO (MC/unid),RESULTADO (MC_total/semana)
4,100097,159.0,220.0,79.0,635.61,630.02,144.73,120.50,23045.30,9519.80,-0.01,-24.23,-13525.50,ABAIXO,NEGATIVO,NEGATIVO
1,100059,80.0,112.0,15.0,577.29,546.28,76.60,51.32,6104.33,769.78,-0.05,-25.28,-5334.55,ABAIXO,NEGATIVO,NEGATIVO
2,100063,57.0,80.0,9.0,340.87,369.32,87.30,101.39,4935.66,912.55,0.08,14.09,-4023.11,ABAIXO,POSITIVO,NEGATIVO
183,101262,78.0,123.0,52.0,419.44,413.81,157.75,168.01,12365.35,8736.64,-0.01,10.26,-3628.71,ABAIXO,POSITIVO,NEGATIVO
106,100763,107.0,135.0,53.0,265.81,274.79,71.90,76.91,7687.36,4076.37,0.03,5.01,-3610.99,ABAIXO,POSITIVO,NEGATIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,120473,1.0,1.0,NaN,78.00,NaN,-0.59,NaN,-0.59,NaN,NaN,NaN,NaN,N/A,N/A,N/A
625,120476,1.0,2.0,NaN,35.41,NaN,0.37,NaN,0.50,NaN,NaN,NaN,NaN,N/A,N/A,N/A
626,120478,1.0,1.0,NaN,28.96,NaN,2.19,NaN,2.19,NaN,NaN,NaN,NaN,N/A,N/A,N/A
627,100867,1.0,2.0,NaN,35.59,NaN,2.65,NaN,3.53,NaN,NaN,NaN,NaN,N/A,N/A,N/A


In [17]:
# Criando um excel com o relatório final
relatorio_final.to_excel(r'relatorio_S17.xlsx', index = False)

# Relatório por canal
##### As operações abaixo são idênticas as operações acima, porém, ao invés de fazer um análise por SKU, será feita uma análise de cada SKU de acordo (agrupado) pelo canal de venda

In [18]:
# Agrupando o 'ecm_delta_calc' por canal
ecm_ref_canal = ecm_delta_calc.groupby(by = ['Código', 'SEMANA', 'MÉDIO_CANAL'], as_index = False).agg(
    vendas_sum = pd.NamedAgg(column = 'Volume / Mês', aggfunc = 'sum'),
    vlr_mean = pd.NamedAgg(column = 'Vr Unitário', aggfunc = 'mean'))

In [19]:
#Gerando as listas para criar o DF
lt_codigos = list()
lt_canal = list()
lt_media_vendas_semana = list()
lt_preco_med_periodo = list()
lt_baseline_min = list()
lt_baseline_max = list()

for cod in ecm_ref_canal['Código'].unique():
    tab_cod = ecm_ref_canal[ecm_ref_canal['Código'] == cod]
    for canal in tab_cod['MÉDIO_CANAL'].unique():
        med_vend = tab_cod[tab_cod['MÉDIO_CANAL'] == canal]['vendas_sum'].mean()
        if tab_cod[tab_cod['MÉDIO_CANAL'] == canal]['vendas_sum'].std() > 0:
            baseline_max = (tab_cod[tab_cod['MÉDIO_CANAL'] == canal]['vendas_sum'].mean() + 
                        tab_cod[tab_cod['MÉDIO_CANAL'] == canal]['vendas_sum'].std())
        else:
            baseline_max = (tab_cod[tab_cod['MÉDIO_CANAL'] == canal]['vendas_sum'].mean())
        med_pc = tab_cod[tab_cod['MÉDIO_CANAL'] == canal]['vlr_mean'].mean()
        lt_codigos.append(cod)
        lt_canal.append(canal)
        lt_baseline_max.append(baseline_max)
        lt_media_vendas_semana.append(med_vend)
        lt_preco_med_periodo.append(med_pc)
        

# Juntando as listas para criar um DF com as informações de vendas do periodo
df_base_canal = pd.DataFrame({'código': lt_codigos, 'canal' : lt_canal, 'base_alvo': lt_media_vendas_semana, 
                        'base_max': lt_baseline_max, 'preço_médio_90dias': lt_preco_med_periodo})

df_base_canal['preço_médio_90dias'] = df_base_canal['preço_médio_90dias'].round(2)
df_base_canal['base_max'] = df_base_canal['base_max'].round(0)
df_base_canal['base_alvo'] = df_base_canal['base_alvo'].round(0)
df_base_canal = df_base_canal.sort_values(by = 'base_alvo', ascending = False)

## DF por canal na semana passada

In [20]:
# Criando um DF com as vendas da semana passada e classificando pelo baseline do período
vendas_semana_passada_canal = ecm_semana.groupby(by = ['Código', 'MÉDIO_CANAL'], as_index=False).agg(
    vendas_semana_passada = pd.NamedAgg(column = 'Volume / Mês', aggfunc = 'sum'),
    minimo_preco_semana = pd.NamedAgg(column = 'Vr Unitário', aggfunc = 'min'),
    maximo_preco_semana = pd.NamedAgg(column = 'Vr Unitário', aggfunc = 'max'))

vendas_semana_passada_canal = vendas_semana_passada_canal.rename(columns={'Código' : 'código', 'MÉDIO_CANAL' : 'canal'})

relatorio_canal = vendas_semana_passada_canal.merge(df_base_canal, how = 'outer', on = ['código', 'canal'])

relatorio_canal['RESULTADO'] = np.where(relatorio_canal['vendas_semana_passada'] >= relatorio_canal['base_max'], 
                                       'ACIMA', np.where(
                                       relatorio_canal['vendas_semana_passada'] <= relatorio_canal['base_alvo'],
                                       'ABAIXO', np.where(
                                           ((relatorio_canal['vendas_semana_passada'] < relatorio_canal['base_max'])
                                            &
                                           (relatorio_canal['vendas_semana_passada'] > relatorio_canal['base_alvo'])),
                                           'ESPERADO', 'N/A')))

In [21]:
#Gerando o relatório final por canal
teste_canal = ecm_semana.groupby(by = ['Código', 'MÉDIO_CANAL'], as_index=False).agg(
    vendas_semana_passada = pd.NamedAgg(column = 'Volume / Mês', aggfunc = 'sum'),
    média_preço_semana = pd.NamedAgg(column = 'Vr Unitário', aggfunc = 'mean'),
    MC_total = pd.NamedAgg(column = 'Margem Contribuição Sem Imposto (R$)', aggfunc = 'sum'))

teste_canal = teste_canal.rename(columns = {'vendas_semana_passada' : 'vendas_s' + ecm_semana['SEMANA'][0],
                                'média_preço_semana' : 'R$_médio_s' + ecm_semana['SEMANA'][0],
                                'MC_total' : 'MC_total_s' + ecm_semana['SEMANA'][0], 'MÉDIO_CANAL' : 'canal', 'Código' : 'código'})

In [22]:
relatorio_t_canal = teste_canal.merge(df_base_canal, how = 'outer', on = ['código', 'canal'])

relatorio_t_canal['RESULTADO (vendas)'] = np.where(relatorio_t_canal[relatorio_t_canal.columns[2]] >= relatorio_canal['base_max'], 
                                       'ACIMA', np.where(
                                       relatorio_t_canal[relatorio_t_canal.columns[2]] <= relatorio_canal['base_alvo'],
                                       'ABAIXO', np.where(
                                           ((relatorio_t_canal[relatorio_t_canal.columns[2]] < relatorio_canal['base_max'])
                                            &
                                           (relatorio_t_canal[relatorio_t_canal.columns[2]] > relatorio_canal['base_alvo'])),
                                           'ESPERADO', 'N/A')))

relatorio_t_canal[relatorio_t_canal.columns[3]] = relatorio_t_canal[relatorio_t_canal.columns[3]].round(2)

relatorio_t_canal['Dif. % R$_médio'] = ((relatorio_t_canal[relatorio_t_canal.columns[3]] / 
                                         relatorio_t_canal['preço_médio_90dias']) - 1).round(2)

relatorio_t_canal['MC_unidade'] = relatorio_t_canal[relatorio_t_canal.columns[4]] / relatorio_t_canal[relatorio_t_canal.columns[2]]
relatorio_t_canal = relatorio_t_canal.rename(columns = {'MC_unidade' : 'MC/unidade_s' + ecm_semana['SEMANA'][0]}).round(2)

In [23]:
# Calculando MC por unidade no período por canal
MC_unid_canal_periodo = ecm_delta_calc.groupby(by = ['Código', 'MÉDIO_CANAL'], as_index=False).agg(
    total_vendas_periodo = pd.NamedAgg(column = 'Volume / Mês', aggfunc = 'sum'),
    total_mc = pd.NamedAgg(column = 'Margem Contribuição Sem Imposto (R$)', aggfunc = 'sum'))

MC_unid_canal_periodo['MC_unitária'] = (MC_unid_canal_periodo['total_mc'] / MC_unid_canal_periodo['total_vendas_periodo']).round(2)
MC_unid_canal_periodo = MC_unid_canal_periodo.rename(columns = {'MÉDIO_CANAL' : 'canal'})
mc_sku_unid_canal = MC_unid_canal_periodo[[ 'Código', 'canal', 'MC_unitária']]
mc_sku_unid_canal = mc_sku_unid_canal.rename(columns={'Código' : 'código'})
relatorio_t_canal = relatorio_t_canal.merge(mc_sku_unid_canal, how = 'outer', on = ['código', 'canal'])
relatorio_t_canal = relatorio_t_canal.rename(columns = {'MC_unitária' : 'MC/unidade_período'})

relatorio_t_canal['Dif_MC/unid'] = relatorio_t_canal[relatorio_t_canal.columns[10]] - relatorio_t_canal['MC/unidade_período']

In [24]:
# Calculando a média da MC total por semana no período por canal
mc_semana_canal = ecm_delta_calc.groupby(by = ['Código', 'SEMANA', 'MÉDIO_CANAL'], as_index=False).agg(
    MC_semana = pd.NamedAgg(column = 'Margem Contribuição Sem Imposto (R$)', aggfunc = 'sum'))

mc_semana_periodo_canal = mc_semana_canal.groupby(by = ['Código', 'MÉDIO_CANAL'], as_index = False).agg(
    qnt_semanas = pd.NamedAgg(column = 'SEMANA', aggfunc = 'count'),
    mc_total_per = pd.NamedAgg(column = 'MC_semana', aggfunc = 'sum'))

mc_semana_periodo_canal['média_mc/semana'] = (mc_semana_periodo_canal['mc_total_per'] / 
                                              mc_semana_periodo_canal['qnt_semanas']).round(2)
media_mc_periodo_canal = mc_semana_periodo_canal[[ 'Código', 'MÉDIO_CANAL', 'média_mc/semana']]

media_mc_periodo_canal = media_mc_periodo_canal.rename(columns = {'Código' : 'código', 'MÉDIO_CANAL' : 'canal'})
relatorio_t_canal = relatorio_t_canal.merge(media_mc_periodo_canal, how = 'outer', on = ['código', 'canal'])
relatorio_t_canal = relatorio_t_canal.rename(columns = {'média_mc/semana' : 'MC/semana_período'})

relatorio_t_canal['Dif. MC/semana'] = relatorio_t_canal[relatorio_t_canal.columns[4]] - relatorio_t_canal['MC/semana_período']

In [25]:
# Gerando 'relatorio_final'
relatorio_t_canal = relatorio_t_canal.rename(columns={'preço_médio_90dias' : 'R$_médio_periodo'})

relatorio_final_canal = relatorio_t_canal [[ 'código', 'canal',  'base_alvo', 'base_max', relatorio_t_canal.columns[2], 
                                'R$_médio_periodo', relatorio_t_canal.columns[3], 
                               'MC/unidade_período', relatorio_t_canal.columns[10],
                               'MC/semana_período', relatorio_t_canal.columns[4],
                               'Dif. % R$_médio', 'Dif_MC/unid', 'Dif. MC/semana', 'RESULTADO (vendas)']]

relatorio_final_canal['RESULTADO (MC/unid)'] = np.where(relatorio_final_canal['Dif_MC/unid'] > 0, 'POSITIVO', np.where(
                                        relatorio_final_canal['Dif_MC/unid'] <= 0, 'NEGATIVO', 'N/A'))

relatorio_final_canal['RESULTADO (MC_total/semana)'] = np.where(relatorio_final_canal['Dif. MC/semana'] > 0, 'POSITIVO', np.where(
                                        relatorio_final_canal['Dif. MC/semana'] <= 0, 'NEGATIVO', 'N/A'))

relatorio_final_canal = relatorio_final_canal.sort_values(by = 'Dif. MC/semana')

<ipython-input-25-db3fe9d36e1c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relatorio_final_canal['RESULTADO (MC/unid)'] = np.where(relatorio_final_canal['Dif_MC/unid'] > 0, 'POSITIVO', np.where(
<ipython-input-25-db3fe9d36e1c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relatorio_final_canal['RESULTADO (MC_total/semana)'] = np.where(relatorio_final_canal['Dif. MC/semana'] > 0, 'POSITIVO', np.where(


In [26]:
relatorio_final_canal.to_excel(r'relatorio_canal.xlsx', index = False)